<a href="https://colab.research.google.com/github/nanpolend/365E5/blob/master/%E9%A0%90%E6%B8%AC%E9%87%91%E5%83%B9%E8%B5%B0%E5%8B%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# -*- coding: utf-8 -*-
"""
黃金價格預測系統（整合技術分析與機器學習）
"""

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ta import add_all_ta_features

class GoldPriceForecaster:
    def __init__(self):
        self.df = None
        self.model = RandomForestRegressor(
            n_estimators=500,
            max_depth=8,
            random_state=42,
            n_jobs=-1
        )
        self.scaler = StandardScaler()

    def fetch_data(self, ticker='GC=F', start_date='2010-01-01'):
        """獲取黃金與關聯市場數據"""
        try:
            # 下載黃金主表（有 OHLCV）
            gold = yf.download(ticker, start=start_date, progress=False)
            for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                gold[col] = gold[col].astype(float)

            # 下載指標，只取 Close 欄位，保證是一維 Series，並對齊 gold
            indicators = {
                'DX-Y.NYB': 'dxy',      # 美元指數
                '^VIX': 'vix',          # 恐慌指數
                '^TNX': 'tnx',          # 10年期美債收益率
                'EURUSD=X': 'eurusd'    # 歐元匯率
            }
            for t, colname in indicators.items():
                data = yf.download(t, start=start_date, progress=False)
                close_series = data['Close']
                # 若還是 DataFrame，取第一欄
                if isinstance(close_series, pd.DataFrame):
                    close_series = close_series.iloc[:, 0]
                # 對齊 gold 的 index，並保證是 float
                gold[colname] = close_series.reindex(gold.index).astype(float)

            self.df = gold.ffill().dropna()
            print(f"數據獲取成功，樣本數：{len(self.df)}")
            return True
        except Exception as e:
            print(f"數據獲取失敗：{str(e)}")
            return False

    def create_features(self, lookback=30):
        """生成技術指標與時間特徵"""
        # 強制所有 OHLCV 欄位為一維 Series
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            arr = np.asarray(self.df[col])
            if arr.ndim > 1:
                arr = arr.ravel()
            self.df[col] = pd.Series(arr, index=self.df.index)

        self.df = add_all_ta_features(
            self.df,
            open="Open", high="High", low="Low",
            close="Close", volume="Volume",
            fillna=True
        )

        self.df['dxy_1y_change'] = self.df['dxy'].pct_change(252)
        self.df['gold_ma_ratio'] = self.df['Close'] / self.df['Close'].rolling(200).mean()
        self.df['day_of_week'] = self.df.index.dayofweek
        self.df['month'] = self.df.index.month
        self.df['target'] = self.df['Close'].pct_change(30).shift(-30)
        self.df.dropna(inplace=True)

        self.feature_cols = [
            'trend_macd', 'momentum_rsi',
            'volatility_bbh', 'volatility_bbl',
            'volume_adi', 'dxy', 'vix',
            'tnx', 'eurusd', 'dxy_1y_change',
            'gold_ma_ratio', 'day_of_week', 'month'
        ]
        print(f"特徵工程完成，使用{len(self.feature_cols)}個特徵")
        # 檢查特徵是否都存在
        for col in self.feature_cols:
            if col not in self.df.columns:
                print(f"警告：缺少特徵 {col}")

    def train_model(self, test_size=0.2):
        X = self.df[self.feature_cols]
        y = self.df['target']
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        self.model.fit(X_train_scaled, y_train)
        preds = self.model.predict(X_test_scaled)
        mae = mean_absolute_error(y_test, preds)
        print(f"模型測試MAE: {mae:.4f} (相當於${mae*self.df['Close'].mean():.2f}波動)")

    def predict_future(self, days=30):
        latest = self.df[self.feature_cols].iloc[-1].values.reshape(1, -1)
        latest_scaled = self.scaler.transform(latest)
        pred_return = self.model.predict(latest_scaled)[0]
        current_price = self.df['Close'].iloc[-1]
        future_price = current_price * (1 + pred_return)
        print(f"\n當前金價: ${current_price:.2f}")
        print(f"預測{days}天後價格: ${future_price:.2f}")
        print(f"預期漲跌幅: {pred_return*100:.1f}%")
        return future_price

    def plot_results(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=self.df.index,
            y=self.df['Close'],
            name='黃金價格',
            line=dict(color='gold')
        ))
        fig.add_trace(go.Scatter(
            x=self.df.index,
            y=self.df['trend_macd'],
            name='MACD',
            line=dict(color='blue'),
            yaxis='y2'
        ))
        fig.update_layout(
            title='黃金價格與技術指標',
            yaxis=dict(title='價格 (USD)'),
            yaxis2=dict(title='MACD', overlaying='y', side='right'),
            hovermode="x unified",
            template="plotly_dark"
        )
        fig.show()

if __name__ == "__main__":
    forecaster = GoldPriceForecaster()
    if forecaster.fetch_data():
        forecaster.create_features()
        forecaster.train_model()
        forecaster.predict_future()
        forecaster.plot_results()

數據獲取成功，樣本數：3844


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)


ValueError: Data must be 1-dimensional, got ndarray of shape (3844, 1) instead